In [10]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
data = pd.read_csv(r'D:\BA prac\HospitalityProject\guest_interactions.csv')

In [13]:
print(data.columns)

Index(['Guest_ID', 'Interaction_Type', 'Interaction_Time', 'Feedback_Score',
       'Room_Number'],
      dtype='object')


In [14]:
# Step 2: Data preprocessing
# Extract relevant columns
texts = data['Feedback_Score'].astype(str).values
labels = data['Interaction_Type'].astype(str).values

In [15]:
# Encode interaction types as numerical labels
interaction_types = list(set(labels))
label_dict = {interaction: i for i, interaction in enumerate(interaction_types)}
encoded_labels = np.array([label_dict[label] for label in labels])

In [16]:
# Step 3: Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)



In [17]:
# Pad sequences for consistent input size
max_sequence_length = 100
X = pad_sequences(sequences, maxlen=max_sequence_length)
y = encoded_labels


In [18]:
# Step 4: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
# Step 5: Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dense(len(interaction_types), activation='softmax'))

c:\Users\Uncle B\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [20]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Step 6: Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.3333 - loss: 1.5984 - val_accuracy: 0.0000e+00 - val_loss: 1.6482
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.0000e+00 - loss: 1.5906 - val_accuracy: 0.0000e+00 - val_loss: 1.6967
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.3333 - loss: 1.5411 - val_accuracy: 0.0000e+00 - val_loss: 1.7524
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.3333 - loss: 1.5025 - val_accuracy: 0.0000e+00 - val_loss: 1.8255
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.3333 - loss: 1.4583 - val_accuracy: 0.0000e+00 - val_loss: 1.9218
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.3333 - loss: 1.4263 - val_accuracy: 0.0000e+00 - val_loss: 2.0496
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.3333 - loss: 1.3879 - val_accuracy: 0.0000e+00 - val_loss: 2.2156
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.6667 - loss: 1.3273 - val_accura

In [22]:
# Step 7: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0000e+00 - loss: 2.7597
Test Accuracy: 0.00


In [23]:
# Step 8: Make predictions
def predict_interaction(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length)
    prediction = model.predict(padded_sequence)
    predicted_label = interaction_types[np.argmax(prediction)]
    return predicted_label

In [24]:
# Example prediction
example_text = "Room was not cleaned properly"
predicted_interaction = predict_interaction(example_text)
print(f'Predicted Interaction Type: {predicted_interaction}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
Predicted Interaction Type: Spa Booking
